In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#install pygaggle before transformers (first create the index using pyserini then install pygaggle)
!pip install pygaggle==0.0.2
!pip install transformers==4.17.0

In [ ]:
import transformers
import pygaggle
import pandas as pd
from tqdm import tqdm
print(transformers.__version__)

4.17.0


In [ ]:
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import DuoT5
reranker =  DuoT5()

In [ ]:
mono=pd.read_csv("/content/drive/MyDrive/Touche/Final_Reranked_psuedo_relevance.csv")
mono.head()

,title_id,title,New_title,doc_id,score,content
0,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-0907wb-60-03588___2,-0.014775,There has been a turnaround in pricing trends ...
1,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-0608wb-66-03868___1,-0.039840,Desktop or Laptop What the Tech HomeForumsCont...
2,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-0608wb-66-03868___2,-0.014093,Laptop computers are portable have the virtual...
3,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-0610wb-81-06832___3,-0.063617,You may even want to contact the instructor ab...
4,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-1904wb-64-15725___1,-0.163953,Desktop or laptop computer IT Donut User login...


In [ ]:
columns_order=mono.columns

In [ ]:
mono.iloc[1]["content"]

'Desktop or Laptop What the Tech HomeForumsContact HijackThisSearchHelp Please visit our forums for help with malware removal or any tech support question Take me to the forums Desktop or Laptop Aug Posted by Hemal in Hardware Do I buy a laptop or a desktop computer for my next machine I am not sure if laptops are just replacing desktops and if I should just not waste my money on a desktop or if they are still better computers Thanks Laptops are becoming more prevalent in the tech world today becoming the staple of computing especially on the college scene Desktops the office work horse are slowly becoming edged out of the picture due to their lack of portability but are they being underrated If you re considering buying either a new desktop or laptop you have to view the pros and cons of both and whether which would serve you better Let s outline a few do you need a laptop or desktop '

In [19]:
new=[]
for title in tqdm(mono["New_title"].unique()):
  passages= mono.loc[mono["New_title"]==title].sort_values(by="score",ascending=False)
  texts = [ Text(p[1].content, {'docid': p[1].doc_id}, p[1].score) for p in passages.iloc[0:50,:].iterrows()]
  query = Query(title)
  reranked = reranker.rerank(query, texts)
  for i in range(0, 50):
    new.append({"doc_id":reranked[i].metadata["docid"], "Score":reranked[i].score, "New_title":title})
  for i in range(50, 1000):
    new.append({"doc_id":passages.iloc[i]["doc_id"], "Score":passages.iloc[i]["score"], "New_title":title})

  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:195: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
100%|██████████| 50/50 [1:21:53<00:00, 98.27s/it]


In [20]:
#Making the reranked file in the same format as intial retrieval file
df=pd.DataFrame(new)
topics=pd.read_csv("/content/drive/MyDrive/Touche/topics-task2-psuedo_relevance.csv")
df=df.join(topics[["Number","Title","New"]].set_index("New"), on="New_title")
df=df.join(mono[["doc_id","content"]].drop_duplicates(subset=['doc_id']).set_index("doc_id"), on="doc_id")
df=df.rename(columns={"Number":"title_id","Score":"score","Title":"title"})
df=df[columns_order]
df

,title_id,title,New_title,doc_id,score,content
0,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-1804wb-90-15876___2,79.028076,Therefore hard drives in a laptop usually have...
1,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-0301wb-84-10147___2,85.443459,Many times consumers may opt to have both but ...
2,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-0608wb-66-03868___2,76.021535,Laptop computers are portable have the virtual...
3,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-0013wb-19-15392___6,74.094092,Keep in mind a laptop is one unit If you break...
4,2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? computer",clueweb12-0907wb-60-03588___2,77.725021,There has been a turnaround in pricing trends ...
...,...,...,...,...,...,...
49995,100,Should I learn Python or R for data analysis?,Should I learn Python or R for data analysis? use,clueweb12-0002wb-53-31513___20,-11.516917,M Tienda Winter Contemporary Japanese Society ...
49996,100,Should I learn Python or R for data analysis?,Should I learn Python or R for data analysis? use,clueweb12-0405wb-12-21320___1,-11.518840,Learn to Program using Python Using Tuples as ...
49997,100,Should I learn Python or R for data analysis?,Should I learn Python or R for data analysis? use,clueweb12-0006wb-18-00098___3,-11.519079,Merry Christmas A Quick Tour of Python The pu...
49998,100,Should I learn Python or R for data analysis?,Should I learn Python or R for data analysis? use,clueweb12-0913wb-49-03622___78,-11.520894,With Examplotron your example XML document is ...


In [22]:
df.to_csv("/content/drive/MyDrive/Touche/Final_Reranked_Duo.csv",index=False)